Analyse exploratoire

# Intro

In [1]:
# Import des bibliothèques
import os
os.chdir("..")
import pandas as pd
import numpy as np
import re
from app.utils.helper import parse_name, give_short_name

# Import des données
from app.data.cleaning import df, athlet, games, region, result

# Athlètes

### Athlètes les plus médaillés tous jeux confondus

In [2]:
# Les 20 athlètes les plus médaillés
n = 20
(athlet
 .join(result.groupby("athlete").sport.first())
 .join(result.groupby("athlete").medal.count())
 .rename(columns={"medal": "n_medals"})
 .sort_values(by="n_medals", ascending=False))[:n]

,first_name,last_name,gender,birth_year,lattest_noc,sport,n_medals
id,,,,,,,
207089493835,Michael,PHELPS,M,1985,USA,Swimming,28
027503151640,Larysa,DIRIY,F,1935,URS,Gymnastics,18
779052604582,Nikolay,ANDRIANOV,M,1953,URS,Gymnastics,15
027629514478,Borys,SHAKHLIN,M,1932,URS,Gymnastics,13
473622409020,Ole,BJRNDALEN,M,1974,NOR,Biathlon,13
206027787982,Edoardo,MANGIAROTTI,M,1919,ITA,Fencing,13
354705440133,Gustaf,CARLBERG,M,1880,SWE,Shooting,13
470290582797,Paavo,NURMI,M,1898,FIN,Athletics,12
429998356546,Sawao,KATO,M,1947,JPN,Gymnastics,12


## Analyse sur la correction des âges (Tokyo 2020)

In [3]:
df[df["ShortName"] == "Leon MARCHAND"]

,Name,Sex,Age,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,FirstName,LastName,ShortName,BirthYear,ApproxBirthYear,AthleteID,GamesID
index,,,,,,,,,,,,,,,,,,,
S230677,MARCHAND Leon,M,19,France,FRA,2020 Summer,2020,Summer,Tokyo,Swimming,Men's 200m Butterfly,NaN,Leon,MARCHAND,Leon MARCHAND,2002,2002,233890689901,S2020
S230678,MARCHAND Leon,M,19,France,FRA,2020 Summer,2020,Summer,Tokyo,Swimming,Men's 200m Individual Medley,NaN,Leon,MARCHAND,Leon MARCHAND,2002,2002,233890689901,S2020
S230679,MARCHAND Leon,M,19,France,FRA,2020 Summer,2020,Summer,Tokyo,Swimming,Men's 400m Individual Medley,NaN,Leon,MARCHAND,Leon MARCHAND,2002,2002,233890689901,S2020
S230680,MARCHAND Leon,M,19,France,FRA,2020 Summer,2020,Summer,Tokyo,Swimming,Men's 4 x 100m Medley Relay Team,NaN,Leon,MARCHAND,Leon MARCHAND,2002,2002,233890689901,S2020


In [9]:
athlet.count()

first_name     147426
last_name      147426
gender         147426
birth_year     141092
lattest_noc    147426
dtype: int64

In [4]:
athlet.count()

first_name     146452
last_name      146452
gender         146452
birth_year     140118
lattest_noc    146452
dtype: int64

In [5]:
# Différence en nombre d'athlètes uniques
147426 - 146452

974

In [6]:
# Différence en nombre d'athlètes uniques
# qui ont une année de naissance renseignée
141092 - 140118

974